 中文语言的文本分类技术和流程，主要包括以下几个步骤：
（1）预处理：去除文本的噪声信息，例如HTML标签，文本格式转换，检测句子边界等等；
（2）中文分词：使用中文分词器为文本分词，并去除停用词；
（3）构建词向量空间：统计文本词频，生成文本的词向量空间；
（4）权重策略--TF-IDF方法：使用TF-IDF发现特征词，并抽取为反映文档主题的特征；
（5）分类器：使用算法训练分类器；
（6）评价分类结果：分类器的测试结果分析。

In [1]:
####################################################################################
################# 文本预处理

# 1.选择处理的文本的范围，2.建立分类文本语料库（训练集和测试集）
# 3.文本格式转换（不同格式的文本不论何种处理形式都要统一转换为纯文本文件）
# 例如：使用lxml库去除html标签

# -*- coding: utf-8 -*-
import sys  
import os
import time
from lxml import etree,html

# htm文件路径，以及读取文件
path = "1.htm"
content = open(path,"rb").read()
page = html.document_fromstring(content) # 解析文件
text = page.text_content() # 去除所有标签
#print(text)              # 输出去除标签后解析结果

# 4.检测句子边界：标记句子的结束。

In [2]:
#####################################################################################
################ 中文分词介绍

#jieba分词（linux环境），它是专门使用Python语言开发的分词系统
import sys  
import os
import jieba

#默认模式
seg_list = jieba.cut("小明1995年毕业于北京清华大学", cut_all=False)
print("Default Mode:", " ".join(seg_list))  

seg_list = jieba.cut("小明1995年毕业于北京清华大学")
print("  ".join(seg_list))

#全模式
seg_list = jieba.cut("小明1995年毕业于北京清华大学", cut_all=True)
print("Full Mode:", "/ ".join(seg_list))  

#搜索引擎模式
seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  
print("/  ".join(seg_list))

ModuleNotFoundError: No module named 'jieba'

In [3]:
#######################################################################################
#对中文语料库进行分词

#设置字符集，并导入jieba分词包
import os 
import jieba

#定义创建两个函数，用于处理读取和保存文件：
#保存至文件
def savefile(savepath,content):
    fp = open(savepath,"wb")
    fp.write(content)
    fp.close()

#读取文件
def readfile(path):
    fp = open(path,"rb")
    content = fp.read()
    fp.close()
    return content

corpus_path = "train_corpus_small/"  # 未分词分类语料库路径
seg_path = "train_corpus_seg/"      # 分词后分类语料库路径

catelist = os.listdir(corpus_path)  # 获取corpus_path下的所有子目录

#接上面的部分，以下是整个语料库的分词主程序：
#获取每个目录下所有的文件
for mydir in catelist:
    class_path = corpus_path+mydir+"/"                 # 拼出分类子目录的路径
    seg_dir = seg_path+mydir+"/"                       # 拼出分词后语料分类目录
    if not os.path.exists(seg_dir):                    # 是否存在目录，如果没有创建
            os.makedirs(seg_dir)
    file_list = os.listdir(class_path)                 # 获取class_path下的所有文件
    
    for file_path in file_list:                       # 遍历类别目录下文件
        fullname = class_path + file_path             # 拼出文件名全路径
        content = readfile(fullname).strip()          # 读取文件内容
        content = content.replace("\r\n".encode('utf-8'),"".encode('utf-8'))          # 删除换行和多余的空格
        content_seg = jieba.cut(content.strip())      # 为文件内容分词
        savefile(seg_dir + file_path," ".join(content_seg).encode('utf-8'))  # 将处理后的文件保存到分词后语料目录

print("中文语料分词结束！！！")

中文语料分词结束！！！


In [20]:
#######################################################################################################
# 在实际应用中，为了后续生成向量空间模型的方便，这些分词后的文本信息还要转换为文本向量信息并对象化。
# 这里需要引入一个Scikit-Learning库的Bunch数据结构：
import sys  
import os 
import pickle
from sklearn.datasets.base import Bunch

# 保存至文件
def savefile(savepath,content):
    fp = open(savepath,"wb")
    fp.write(content).encode('utf-8')
    fp.close()

# 读取文件	
def readfile(path):
    fp = open(path,"rb")
    content = fp.read().decode('utf-8')
    fp.close()
    return content


# Bunch类提供一种key,value的对象形式
# target_name: 所有分类集名称列表
# label:       每个文件的分类标签列表
# filenames:   文件路径
# contents:    分词后文件词向量形式
bunch = Bunch(target_name=[],label=[],filenames=[],contents=[])	

wordbag_path = "test_word_bag/test_set.dat"  # 持久化分词分类语料库路径
seg_path = "test_corpus_seg/"                # 分词后分类语料库路径


catelist = os.listdir(seg_path)              # 获取seg_path下的所有子目录
bunch.target_name.extend(catelist)
# 获取每个目录下所有的文件
for mydir in catelist:
    class_path = seg_path+mydir+"/"            # 拼出分类子目录的路径
    file_list = os.listdir(class_path)         # 获取class_path下的所有文件
    for file_path in file_list:               # 遍历类别目录下文件
        fullname = class_path + file_path      # 拼出文件名全路径
        bunch.label.append(mydir)
        bunch.filenames.append(fullname)
        bunch.contents.append(readfile(fullname).strip())  # 读取文件内容
# 函数原型
# 声明：s为字符串，rm为要删除的字符序列
# s.strip(rm)   删除s字符串中开头、结尾处，位于rm删除序列的字符
# s.lstrip(rm)  删除s字符串中开头处，位于 rm删除序列的字符
# s.rstrip(rm)  删除s字符串中结尾处，位于 rm删除序列的字符
# 当rm为空时，默认删除空白符（包括'\n', '\r',  '\t',  ' ')

#对象持久化                                                                                              
file_obj = open(wordbag_path, "wb")
pickle.dump(bunch,file_obj)                      
file_obj.close()

print('target_name:',bunch.target_name)
print('label:',bunch.label[5:15])
print('contents:',bunch.contents[1][0:10])
print('',bunch.filenames[1])
print("构建文本对象结束！！！")

target_name: ['art', 'computer', 'economic', 'education', 'environment', 'medical', 'military', 'politics', 'sports', 'traffic']
label: ['art', 'art', 'art', 'art', 'art', 'art', 'computer', 'computer', 'computer', 'computer']
contents: ﻿ 唐建平 出处 ：
 test_corpus_seg/art/2160.txt
构建文本对象结束！！！


In [21]:
##########################################################################################################
###################    停用词
# 由于文本在存储为向量空间时，维度比较高。为节省存储空间和提高搜索效率，在文本分类之前会自动
# 过滤掉某些字或词，这些字或词即被称为停用词
# 从这个网址下载停用词表：http://www.threedweb.cn/thread-1294-1-1.html
# 读取停用词列表
# 读取文件 
# 1. 读取停用词表 
# 读取文件	

stopword_path = "train_word_bag/hlt_stop_words.txt" 
# Python splitlines() 按照行('\r', '\r\n', \n')分隔，返回一个包含各行作为元素的列表，
# 如果参数 keepends 为 False，不包含换行符，如果为 True，则保留换行符。
stpwrdlst = readfile(stopword_path).splitlines()
print(stpwrdlst[10:20])

['*', '一一', '~~~~', '’', '. ', '『', '.一', './', '-- ', '』']


In [22]:
#############################################################################################################
# 权重策略：TF-IDF方法（Tf–idf term weighting）
# 在一个大的文本语料库中，一些单词将会非常的出现（例如“the”，“a”，“is”是英文），因此对文档的实际内容没有
# 任何有意义的信息.如果我们将直接计数数据直接提供给分类器，那么非常频繁的词语会影向有意义词语的频率。
# 为了将计数特征重新调整为适合分类器使用的浮点值，使用tf-idf变换是非常常见的。

# 词频（term frequency，TF）指的是某一个给定的词语在该文件中出现的频率。
# 逆向文件频率（inverse document frequency，IDF）是一个词语普遍重要性的度量。
# 某一特定词语的IDF，可以由总文件数目除以包含该词语之文件的数目，再将得到的商取对数得到。

# Tf表示词语频率，而tf-idf表示术语频率乘以逆文档频率：
# tf-idf(t,d) = tf(t,d) x idf(t)

# coding: utf-8

import sys  
import os 
#引入Bunch类
from sklearn.datasets.base import Bunch
#引入持久化类
import pickle
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import TfidfVectorizer 
# 读取文件
def readfile(path):
    fp = open(path,"rb")
    content = fp.read()
    fp.close()
    return content

#计算训练语料的tf-idf权值并持久化为词袋

#读取bunch对象
def readbunchobj(path):
    file_obj = open(path, "rb")
    bunch = pickle.load(file_obj)
    file_obj.close()
    return bunch
#写入bunch对象	
def writebunchobj(path,bunchobj):
    file_obj = open(path, "wb")
    pickle.dump(bunchobj,file_obj) 
    file_obj.close()

# 1. 读取停用词表	
stopword_path = "train_word_bag/hlt_stop_words.txt"
stpwrdlst = readfile(stopword_path).splitlines()

# 2. 导入分词后的词向量bunch对象
path = "test_word_bag/test_set.dat"        # 词向量空间保存路径
bunch = readbunchobj(path)

# 3. 构建测试集tf-idf向量空间
tfidfspace = Bunch(target_name=bunch.target_name,label=bunch.label,filenames=bunch.filenames, tdm=[], vocabulary={})

# 4. 使用TfidfVectorizer初始化向量空间模型 
#    TfidfVectorizer的类，它将CountVectorizer和TfidfTransformer的所有选项组合在一个模型中
vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,max_df = 0.5)
# 该类会统计每个词语的tf-idf权值
transformer = TfidfTransformer() 

#5. 文本转为词频矩阵,单独保存字典文件 
tfidfspace.tdm = vectorizer.fit_transform(bunch.contents)
tfidfspace.vocabulary = vectorizer.vocabulary_
print('训练词频稀疏矩阵:',tfidfspace.tdm.toarray()[0:5])
print('训练词汇字典:',list(tfidfspace.vocabulary.items())[0:10])

#6. 创建词袋的持久化
space_path = "train_word_bag/tfdifspace.dat"        # 词向量空间保存路径
writebunchobj(space_path,tfidfspace)

print("if-idf词向量空间创建成功！！！")

训练词频稀疏矩阵: [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
训练词汇字典: [('19960518', 322), ('努力', 8526), ('创作', 8087), ('无愧于', 16791), ('伟大', 6119), ('时代', 16942), ('艺术', 23812), ('精品', 22397), ('刘忠德', 8064), ('艺术创作', 23816)]
if-idf词向量空间创建成功！！！


In [7]:
# 训练步骤与训练集相同，首先是分词，之后生成文件词向量文件，直至生成词向量模型。
# 不同的是在训练词向量模型时，需要加载训练集词袋，将测试集产生的词向量映射到训练
# 集词袋的词典中，生成向量空间模型。

import sys  
import os 
#引入Bunch类
from sklearn.datasets.base import Bunch
#引入持久化类
import pickle
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import TfidfVectorizer  


# 读取文件
def readfile(path):
    fp = open(path,"rb")
    content = fp.read()
    fp.close()
    return content

#计算训练语料的tfidf权值并持久化为词袋

#读取bunch对象
def readbunchobj(path):
    file_obj = open(path, "rb")
    bunch = pickle.load(file_obj) 
    file_obj.close()
    return bunch
#写入bunch对象	
def writebunchobj(path,bunchobj):
    file_obj = open(path, "wb")
    pickle.dump(bunchobj,file_obj) 
    file_obj.close()

# 1. 读取停用词表	
stopword_path = "train_word_bag/hlt_stop_words.txt"
stpwrdlst = readfile(stopword_path).splitlines()

# 2. 导入分词后的词向量bunch对象
path = "test_word_bag/test_set.dat"        # 词向量空间保存路径
bunch = readbunchobj(path)

# 3. 构建测试集tfidf向量空间
testspace = Bunch(target_name=bunch.target_name,label=bunch.label,filenames=bunch.filenames,tdm=[],vocabulary={})

# 4. 导入训练集的词袋
trainbunch = readbunchobj("train_word_bag/tfdifspace.dat")

# 5. 使用TfidfVectorizer初始化向量空间模型 
vectorizer = TfidfVectorizer(stop_words=stpwrdlst,sublinear_tf = True,max_df = 0.5,vocabulary=trainbunch.vocabulary)
transformer=TfidfTransformer() # 该类会统计每个词语的tf-idf权值

# 文本转为tf-idf矩阵,单独保存字典文件 
testspace.tdm = vectorizer.fit_transform(bunch.contents)
testspace.vocabulary = trainbunch.vocabulary
print('测试词频稀疏矩阵:',testspace.tdm.toarray()[0:5])
# 测试集词汇就是训练集词汇
print('测试词汇字典:',list(testspace.vocabulary.items())[0:10])
print('测试词汇字典:',list(vectorizer.vocabulary_.items())[0:10])

# 创建词袋的持久化
space_path = "test_word_bag/testspace.dat"        # 词向量空间保存路径
writebunchobj(space_path,testspace)

print("test词向量空间创建成功！！！")

测试词频稀疏矩阵: [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
测试词汇字典: [('19960518', 322), ('努力', 8526), ('创作', 8087), ('无愧于', 16791), ('伟大', 6119), ('时代', 16942), ('艺术', 23812), ('精品', 22397), ('刘忠德', 8064), ('艺术创作', 23816)]
测试词汇字典: [('19960518', 322), ('努力', 8526), ('创作', 8087), ('无愧于', 16791), ('伟大', 6119), ('时代', 16942), ('艺术', 23812), ('精品', 22397), ('刘忠德', 8064), ('艺术创作', 23816)]
test词向量空间创建成功！！！


In [23]:
# 执行多项式贝叶斯算法进行测试文本分类，并返回分类精度：
import sys  
import os 
#引入Bunch类
from sklearn.datasets.base import Bunch
#引入持久化类
import pickle
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.naive_bayes import MultinomialNB #导入多项式贝叶斯算法
import numpy as np
from sklearn import metrics


# 读取文件
def readfile(path):
    fp = open(path,"rb")
    content = fp.read()
    fp.close()
    return content

#计算分类精度：
def metrics_result(actual,predict):
    print ('精度:{0:.3f}'.format(metrics.precision_score(actual,predict,average = 'weighted'))) 
    print ('召回:{0:0.3f}'.format(metrics.recall_score(actual,predict,average = 'weighted')))  
    print ('f1-score:{0:.3f}'.format(metrics.f1_score(actual,predict,average = 'weighted')))  

#读取bunch对象
def readbunchobj(path):
    file_obj = open(path, "rb")
    bunch = pickle.load(file_obj) 
    file_obj.close()
    return bunch

#写入bunch对象
def writebunchobj(path,bunchobj):
    file_obj = open(path, "wb")
    pickle.dump(bunchobj,file_obj) 
    file_obj.close()

# 导入训练集	
trainpath = "train_word_bag/tfdifspace.dat"
train_set = readbunchobj(trainpath)

# 导入测试集
testpath = "test_word_bag/testspace.dat"
test_set = readbunchobj(testpath)
# 应用朴素贝叶斯算法 
# 1. 输入词袋向量和分类标签
#alpha:0.001 alpha越小，迭代次数越多，精度越高
clf = MultinomialNB(alpha = 0.001).fit(train_set.tdm, train_set.label)

# 预测分类结果
predicted = clf.predict(test_set.tdm)
total = len(predicted);rate = 0
for flabel,file_name,expct_cate in zip(test_set.label,test_set.filenames,predicted):
    if flabel != expct_cate:
        rate += 1
        print(file_name,": 实际类别:",flabel," -->预测类别:",expct_cate)
# 精度
print("error rate:",float(rate)*100/float(total),"%")
print("预测完毕!!!")

metrics_result(test_set.label,predicted)

test_corpus_seg/art/3143.txt : 实际类别: art  -->预测类别: education
error rate: 0.9900990099009901 %
预测完毕!!!
精度:0.991
召回:0.990
f1-score:0.990
